In [ ]:
import os
import numpy as np
import pandas as pd
import json

from parallel_robust_forest import load_attack_rules
from sklearn.model_selection import train_test_split
from joblib import Parallel,delayed

# Functions

In [ ]:
# Functions

def load_dataset(path, 
                 dataset_filename, 
                 sep=",", 
                 header=None,
                 nrows=None,
                 names=None, 
                 index_col=False, 
                 na_values='?'):
    
    return pd.read_csv(os.path.join(path,dataset_filename), 
                       sep=sep, 
                       header=header,
                       nrows=nrows,
                       names=names, 
                       index_col=index_col, 
                       na_values='?')

def binarize_data(data, label, threshold):
    data[label] = np.where(data[label] >= threshold, 1, -1)

def serialize_dataset(p_data, 
                      path, 
                      dataset_filename, 
                      suffix, 
                      sep=",", 
                      compression="bz2", 
                      index=False):
    
    p_data.to_csv(path+"/"+dataset_filename.split(".")[0]+suffix, 
                  sep=sep, 
                  compression=compression, 
                  index=index)

In [12]:
def perturb_instance(x, rules, budget, max_budget_per_feature, thresholds):
    """
    Returns the set of possible perturbations of a given instance.

    This function takes as input an instance and returns a set of perturbations of that instance, 
    using the specified amount of budget and considering the cost of perturbing each individual feature.

    Parameters
    ----------
    x : pandas.Series
        The original instance
    rules : list
        The list of modification rules
    budget : float
        The attacker's budget
    max_budget_per_feature : dict
        The maximum allowed amount of budget units that can be spend on each feature
    thresholds : dict
        feature -> list of relevant thresholds

    Returns
    -------
    pandas.DataFrame
        The set of perturbations (including the original instance, placed at the very beginning)
    """
    
    # initialize the queue (FIFO) with both the original instance, 
    # the initial budget, and an empty dictionary of budget units spent so far
    queue = [(x, budget, {})]
    # visited perturbations
    seen = { tuple(x): [budget, {}] }
    # initialize the set of perturbations of this instance with the empty list
    # perturbations = []
    
    # loop until the queue is not empty
    while len(queue)>0:
        item = queue.pop() # dequeue the first inserted element
        x = item[0] # get the instance
        b = item[1] # get the residual budget
        budget_units_spent = item[2] # get the dictionary containing the amount of budget spent on each feature, so far
        
        # loop through all the features subject to the set of attack rules
        for r in rules:
            f = x.index.values[r.get_target_feature()]
            # check budget
            if not( r.get_cost() <= b and 
                   budget_units_spent.get(f, 0) + r.get_cost() <= max_budget_per_feature[f] ):
                continue
            # check validity
            if not r.is_applicable(x):
                continue
            
            # apply rule to a copy
            x_atks = []
            if not r.is_num():
                xx = r.apply(x)
                x_atks += [xx]
            else:
                x_tmp = r.apply(x)
                # Evaluate crossing of multiple thresholds
                low,high=sorted([x[f], x_tmp[f]])
                z = set(thresholds[f][np.logical_and(thresholds[f]>=low, thresholds[f]<=high)])
                z |= set([low,high])
                # Evaluate extremes of validity interval
                extremes = r.get_pre_interval()
                z |= set([t for t in extremes if low < t < high])
                
                for zi in z:
                    xx = x.copy()
                    xx[f] = zi
                    x_atks += [xx]

            # process all atks
            for xx in x_atks:
                # skip if already seen and with a larger residual budget
                xx_t = tuple(xx)
                res_b = b - r.get_cost()
                seen_budgets = seen.get(xx_t)
                if seen_budgets is not None and seen_budgets[0] >= res_b:
                    continue

                # update budgets spent
                updated_budget_units_spent = budget_units_spent.copy()
                updated_budget_units_spent[f] = updated_budget_units_spent.get(f,0) + r.get_cost()
                # add to frontier and to past seen elements
                seen[xx_t] = [res_b, updated_budget_units_spent]
                queue.append([xx, res_b, updated_budget_units_spent])
    
    perturbations_df = pd.DataFrame.from_records(list(seen.keys()), columns=x.index.values)
    perturbations_df = perturbations_df.drop_duplicates()
    return perturbations_df

In [13]:
def perturb_thread(x, rules, budget, max_budget_per_feature, thresholds,skip_class,instance_id =-1):
    
    if instance_id%500==0:
        print("***** Perturbing instance [ID = #{}]... *****".format(instance_id))  
        
    if skip_class is not None and instance[-1]==skip_class:
        # keep the original instance only
        perturbations = pd.DataFrame([x])
    else:
            
        perturbations = perturb_instance(x=x, rules=rules, budget=budget, 
                                             max_budget_per_feature=max_budget_per_feature,
                                             thresholds=thresholds) 
            
    perturbations.insert(loc=0,  column="instance_id", 
                            value=[instance_id for i in range(perturbations.shape[0])], 
                            allow_duplicates=True)
    return perturbations

In [14]:
def perturb_dataset(data, budget, max_budget_per_feature, rules, skip_class=None):
    """
    Returns the dataset extended with all instance perturbations.

    This function takes as input a dataset and returns another dataset which is obtained from the original
    by adding all the possible perturbations an attacker with budget B can apply to every instance.

    Parameters
    ----------
    data : pandas.DataFrame
        The original dataset
    rules : list
        The list of modification rules
    budget : float
        The attacker's budget
    max_budget_per_feature : dict
        The maximum allowed amount of budget units that can be spend on each feature
    costs : dict
        A mapping between each feature and its cost of perturbation
    skip_class : int
        if class (i.e. last columns) equals skip_class, then instance is skipped

    Returns
    -------
    pandas.DataFrame
        The perturbed dataset

    """
    if data is None or data.empty:
        return # if not, just return None
    
    # compute valid thresholds
    thresholds = {c:np.unique(data[c]) for c in data.columns}

    # prepare the perturbed dataset to be returned, initially empty with an extra "instance_id" column
    cols = ["instance_id"] + data.columns.tolist()
    perturbed_data = pd.DataFrame(columns=cols)
    
    perturbed_data = Parallel(n_jobs=16)(delayed(perturb_thread)
                                             (x=instance, rules=rules, budget=budget,
                                                max_budget_per_feature=max_budget_per_feature,
                                                thresholds=thresholds, skip_class=skip_class,
                                                instance_id=instance_id+1) # start from id 1
                                        for instance_id,(index, instance) in enumerate(data.iterrows()))
    
    perturbed_data = pd.concat(perturbed_data)
    
    # eventually, return the perturbed dataset
    print("***** Return the final perturbed dataset *****")
    
    return perturbed_data

# WINE Dataset

In [15]:
DATASET_NAME="wine"

raw_colnames = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide',
            'total_sulfur_dioxide', 'density', 'pH', 'sulphites', 'alcohol', 'is_white', 'quality']

DATASET_PATH="../data/{}".format(DATASET_NAME)
RAW_DATASET_PATH=DATASET_PATH + "/raw"
ATK_DATASET_PATH=DATASET_PATH + "/attacks"

ATKS_FILE = os.path.join(ATK_DATASET_PATH, "attacks.json")

TRAINING_SET="train.csv.bz2"
VALIDATION_SET="valid.csv.bz2"
TEST_SET="test.csv.bz2"
N_TRAIN_INSTANCES=None # replace this with None to load the whole training set
N_TEST_INSTANCES=None # replace this with None to load the whole test set

In [16]:
# load
train = load_dataset(RAW_DATASET_PATH, TRAINING_SET, names=raw_colnames, header=0, nrows=N_TRAIN_INSTANCES)
valid = load_dataset(RAW_DATASET_PATH, VALIDATION_SET, names=raw_colnames, header=0)
test  = load_dataset(RAW_DATASET_PATH, TEST_SET, names=raw_colnames, header=0, nrows=N_TEST_INSTANCES)

# binarize
binarize_data(train, "quality", 6)
binarize_data(valid, "quality", 6)
binarize_data(test, "quality", 6)

print("Shape of training set: {}".format(train.shape))
print("Shape of validation set: {}".format(valid.shape))
print("Shape of test set: {}".format(test.shape))

serialize_dataset(train, DATASET_PATH, TRAINING_SET, ".csv.bz2")
serialize_dataset(valid, DATASET_PATH, VALIDATION_SET, ".csv.bz2")
serialize_dataset(test, DATASET_PATH, TEST_SET, ".csv.bz2")

# train.head()
# valid.head()
# test.head()

Shape of training set: (4547, 13)
Shape of validation set: (650, 13)
Shape of test set: (1300, 13)


In [19]:
## WITH ATTACKS

######################
# Attacker Definition
######################

max_budget_per_feature = {
    'alcohol'          : 1000,
    'residual_sugar'   : 1000,
    'volatile_acidity' : 1000
}

attacker_rules = load_attack_rules(ATKS_FILE, list(train.columns.values) )

B = [30, 60]


for budget in B:
    print ("Processing Budget: ", budget)
    train_att = perturb_dataset(train, budget, max_budget_per_feature, attacker_rules, skip_class=None)
    valid_att = perturb_dataset(valid, budget, max_budget_per_feature, attacker_rules, skip_class=None)
    test_att  = perturb_dataset(test, budget, max_budget_per_feature, attacker_rules, skip_class=None)
    
    serialize_dataset(train_att, ATK_DATASET_PATH, TRAINING_SET, "_B{}".format(budget)+".atks.bz2")
    serialize_dataset(valid_att, ATK_DATASET_PATH, VALIDATION_SET, "_B{}".format(budget)+".atks.bz2")
    serialize_dataset(test_att, ATK_DATASET_PATH, TEST_SET, "_B{}".format(budget)+".atks.bz2")
    
    print("Shape of attacked training set: {}".format(train_att.shape))
    print("Shape of attacked validation set: {}".format(valid_att.shape))
    print("Shape of attacked test set: {}".format(test_att.shape))

***** Return the final perturbed dataset *****


In [20]:
train_att

,instance_id,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphites,alcohol,is_white,quality
0,1,8.9,0.59,0.39,2.3,0.095,5.0,22.0,0.99860,3.37,0.58,10.30,0.0,-1.0
0,2,6.8,0.27,0.12,1.3,0.040,87.0,168.0,0.99200,3.18,0.41,10.00,1.0,-1.0
0,3,6.3,0.31,0.34,2.2,0.045,20.0,77.0,0.99270,3.30,0.43,10.20,1.0,-1.0
0,4,7.8,0.29,0.22,9.5,0.056,44.0,213.0,0.99715,3.08,0.61,9.30,1.0,1.0
1,4,7.8,0.29,0.22,9.5,0.056,44.0,213.0,0.99715,3.08,0.61,10.05,1.0,1.0
2,4,7.8,0.29,0.22,9.5,0.056,44.0,213.0,0.99715,3.08,0.61,9.99,1.0,1.0
3,4,7.8,0.29,0.22,9.5,0.056,44.0,213.0,0.99715,3.08,0.61,10.00,1.0,1.0
4,4,7.8,0.29,0.22,9.5,0.056,44.0,213.0,0.99715,3.08,0.61,9.80,1.0,1.0
5,4,7.8,0.29,0.22,8.3,0.056,44.0,213.0,0.99715,3.08,0.61,9.30,1.0,1.0
6,4,7.8,0.29,0.22,8.3,0.056,44.0,213.0,0.99715,3.08,0.61,10.05,1.0,1.0


# CENSUS Dataset

In [ ]:
DATASET_NAME="census"

raw_colnames = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
                'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 
                'hours_per_week', 'native_country', 'income_greater_than_50k']

DATASET_PATH="../data/{}".format(DATASET_NAME)
RAW_DATASET_PATH=DATASET_PATH + "/raw"
ATK_DATASET_PATH=DATASET_PATH + "/attacks"

ATKS_FILE = os.path.join(ATK_DATASET_PATH, "attacks.json")

TRAINING_SET="train.csv.bz2"
VALIDATION_SET="valid.csv.bz2"
TEST_SET="test.csv.bz2"

N_TRAIN_INSTANCES=None # replace this with None to load the whole training set
N_TEST_INSTANCES=None # replace this with None to load the whole test set



In [ ]:
## WITHOUT ATTACKS

# load
train = load_dataset(RAW_DATASET_PATH, TRAINING_SET, names=raw_colnames, header=0, nrows=N_TRAIN_INSTANCES)
test = load_dataset(RAW_DATASET_PATH, TEST_SET, names=raw_colnames, header=0, nrows=N_TEST_INSTANCES)

# remove education string
train = train.drop(['education'], axis=1)
test  = test.drop(['education'], axis=1)
# drop NA
train = train[~train.isnull().any(axis=1)]
test  = test[~test.isnull().any(axis=1)]

# create the missing validation set
train, valid = train_test_split( train, 
                                 test_size=0.1, 
                                 random_state=42, 
                                 stratify=train["income_greater_than_50k"])

print("Shape of training set: {}".format(train.shape))
print("Shape of validation set: {}".format(valid.shape))
print("Shape of test set: {}".format(test.shape))

serialize_dataset(train, DATASET_PATH, TRAINING_SET, ".csv.bz2")
serialize_dataset(valid, DATASET_PATH, VALIDATION_SET, ".csv.bz2")
serialize_dataset(test, DATASET_PATH, TEST_SET, ".csv.bz2")


# train.head()
# valid.head()
# test.head()

In [ ]:
## WITH ATTACKS

######################
# Attacker Definition
######################
max_budget_per_feature = {
    'workclass'     : 90,
    'marital_status': 90,
    'occupation'    : 90,
    'education_num' : 90,
    'hours_per_week': 90,
    'capital_gain'  : 90
}

B = [30] # 90

attacker_rules = load_attack_rules(ATKS_FILE, list(train.columns.values) )

for budget in B:
    print ("Processing Budget: ", budget)
    train_att = perturb_dataset(train, budget, max_budget_per_feature, attacker_rules, skip_class=None)
    valid_att = perturb_dataset(valid, budget, max_budget_per_feature, attacker_rules, skip_class=None)
    test_att  = perturb_dataset(test, budget, max_budget_per_feature, attacker_rules, skip_class=None)
    
    serialize_dataset(train_att, ATK_DATASET_PATH, TRAINING_SET, "_B{}".format(budget)+".atks.bz2")
    serialize_dataset(valid_att, ATK_DATASET_PATH, VALIDATION_SET, "_B{}".format(budget)+".atks.bz2")
    serialize_dataset(test_att, ATK_DATASET_PATH, TEST_SET, "_B{}".format(budget)+".atks.bz2")
    
    print("Shape of attacked training set: {}".format(train_att.shape))
    print("Shape of attacked validation set: {}".format(valid_att.shape))
    print("Shape of attacked test set: {}".format(test_att.shape))

# SPAM TO BE FIXED

In [ ]:
DATASET_NAME="spam2"

colnames = ['word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d',
       'word_freq_our', 'word_freq_over', 'word_freq_remove',
       'word_freq_internet', 'word_freq_order', 'word_freq_mail',
       'word_freq_receive', 'word_freq_will', 'word_freq_people',
       'word_freq_report', 'word_freq_addresses', 'word_freq_free',
       'word_freq_business', 'word_freq_email', 'word_freq_you',
       'word_freq_credit', 'word_freq_your', 'word_freq_font', 'word_freq_000',
       'word_freq_money', 'word_freq_hp', 'word_freq_hpl', 'word_freq_george',
       'word_freq_650', 'word_freq_lab', 'word_freq_labs', 'word_freq_telnet',
       'word_freq_857', 'word_freq_data', 'word_freq_415', 'word_freq_85',
       'word_freq_technology', 'word_freq_1999', 'word_freq_parts',
       'word_freq_pm', 'word_freq_direct', 'word_freq_cs', 'word_freq_meeting',
       'word_freq_original', 'word_freq_project', 'word_freq_re',
       'word_freq_edu', 'word_freq_table', 'word_freq_conference',
       'char_freq_;', 'char_freq_(', 'char_freq_[', 'char_freq_!',
       'char_freq_$', 'char_freq_#', 'capital_run_length_average',
       'capital_run_length_longest', 'capital_run_length_total', 'spam']

DATASET_PATH="../data/{}".format(DATASET_NAME)
RAW_DATASET_PATH=DATASET_PATH + "/raw"
ATK_DATASET_PATH=DATASET_PATH + "/attacks"

TRAINING_SET="train.csv.bz2"
VALIDATION_SET="valid.csv.bz2"
TEST_SET="test.csv.bz2"
N_TRAIN_INSTANCES=None # replace this with None to load the whole training set
N_TEST_INSTANCES=None # replace this with None to load the whole test set

######################
# Attacker Definition
######################
attacker_rules = [
    {'f'    : 'char_freq_!',
     'valid': lambda x: x['char_freq_!']>=1,
     'value': -1,
     'cost' : 15,
     'is_cat': False },
    {'f'    : 'word_freq_remove',
     'valid': lambda x: x['word_freq_remove']>=.5,
     'value': -0.5,
     'cost' : 15,
     'is_cat': False },
    {'f'    : 'char_freq_$',
     'valid': lambda x: x['char_freq_$']>=.1,
     'value': -0.1,
     'cost' : 15,
     'is_cat': False },    
    {'f'    : 'capital_run_length_average',
     'valid': lambda x: x['capital_run_length_average']>=5,
     'value': -1,
     'cost' : 30,
     'is_cat': False }, 
    {'f'    : 'capital_run_length_total',
     'valid': lambda x: x['capital_run_length_total']>=400,
     'value': -50,
     'cost' : 30,
     'is_cat': False }
#     , 
#     {'f'    : 'word_freq_hp',
#      'valid': lambda x: x['word_freq_hp']>=1,
#      'value': -1,
#      'cost' : 30,
#      'is_cat': False }
]

max_budget_per_feature = {
    'char_freq_!'     : 1000,
    'word_freq_remove': 1000,
    'char_freq_$'     : 1000,
    'capital_run_length_average' : 1000,
    'capital_run_length_total': 1000,
    'word_freq_hp'  : 1000
}

B = [30, 60]

In [ ]:
# load
train = load_dataset(RAW_DATASET_PATH, TRAINING_SET, names=colnames, header=0, nrows=N_TRAIN_INSTANCES)
valid = load_dataset(RAW_DATASET_PATH, VALIDATION_SET, names=colnames, header=0)
test = load_dataset(RAW_DATASET_PATH, TEST_SET, names=colnames, header=0, nrows=N_TEST_INSTANCES)

# binarize
binarize_data(train, "spam", 0.5)
binarize_data(valid, "spam", 0.5)
binarize_data(test, "spam", 0.5)

print("Shape of training set: {}".format(train.shape))
print("Shape of validation set: {}".format(valid.shape))
print("Shape of test set: {}".format(test.shape))

# train.head()
# valid.head()
# test.head()

In [ ]:
serialize_dataset(train, DATASET_PATH, TRAINING_SET, ".csv.bz2")
serialize_dataset(valid, DATASET_PATH, VALIDATION_SET, ".csv.bz2")
serialize_dataset(test, DATASET_PATH, TEST_SET, ".csv.bz2")

In [ ]:
for budget in B:
    print ("Processing Budget: ", budget)
    train_att = perturb_dataset(train, budget, max_budget_per_feature, attacker_rules, skip_class=None)
    valid_att = perturb_dataset(valid, budget, max_budget_per_feature, attacker_rules, skip_class=None)
    test_att  = perturb_dataset(test, budget, max_budget_per_feature, attacker_rules, skip_class=None)
    
    serialize_dataset(train_att, ATK_DATASET_PATH, TRAINING_SET, "_B{}".format(budget)+".atks.bz2")
    serialize_dataset(valid_att, ATK_DATASET_PATH, VALIDATION_SET, "_B{}".format(budget)+".atks.bz2")
    serialize_dataset(test_att, ATK_DATASET_PATH, TEST_SET, "_B{}".format(budget)+".atks.bz2")
    
    print("Shape of attacked training set: {}".format(train_att.shape))
    print("Shape of attacked validation set: {}".format(valid_att.shape))
    print("Shape of attacked test set: {}".format(test_att.shape))